
<a id='optimization-solver-packages'></a>
<div id="qe-notebook-header" style="text-align:right;">
        <a href="https://quantecon.org/" title="quantecon.org">
                <img style="width:250px;display:inline;" src="https://assets.quantecon.org/img/qe-menubar-logo.svg" alt="QuantEcon">
        </a>
</div>

# Solucionadores, Otimizadores, e Diferenciação Automática

## Conteúdo

- [Solucionadores, Otimizadores, e Diferenciação Automática](#Solucionadores,-Otimizadores,-e-Diferenciação-Automática)  
  - [Resumo](#Resumo)  
  - [Introdução a Diferenciação Automática](#Introdução-a-Diferenciação-Automática)  
  - [Otimização](#Otimização)  
  - [Sistema de Equações e Mínimos Quadrados](#Sistema-de-Equações-e-Mínimos-Quadrados)  
  - [LeastSquaresOptim.jl](#LeastSquaresOptim.jl)  
  - [Notas Adicionais](#Notas-Adicionais)  
  - [Exercícios](#Exercícios)

> *Devidamente traduzido, revisado e adaptado do [QuantEcon](https://quantecon.org/) pelos bolsistas CNPq, Pedro Luiz H. Furtado e Jonas Aragão M. Corpes, sob supervisão do Prof. Christiano Penna, do CAEN/UFC.*

## Resumo

Nesta aula, apresentamos algumas das bibliotecas do Julia que consideramos particularmente úteis para trabalhos quantitativos em economia.

### Configuração

In [1]:
using InstantiateFromURL
github_project("QuantEcon/quantecon-notebooks-julia", version = "0.5.0")
# github_project("QuantEcon/quantecon-notebooks-julia", version = "0.5.0", instantiate = true) # uncomment to force package installation

In [2]:
using LinearAlgebra, Statistics
using ForwardDiff, Zygote, Optim, JuMP, Ipopt, BlackBoxOptim, Roots, NLsolve, LeastSquaresOptim
using Optim: converged, maximum, maximizer, minimizer, iterations #algumas funções extras

## Introdução a Diferenciação Automática

A promessa de uma programação diferenciável é que podemos avançar para obter derivadas quase arbitrariamente cpmplicadas em
programas de computador, em vez de simplesmente pensar nas derivadas das funções matemáticas. A programação diferenciável é a evolução natural da diferenciação automática (DA, às vezes chamada de diferenciação algorítmica).

Recuando, existem três maneiras de calcular o gradiente ou o efeito jacobiano:

- Derivadas analíticas / Diferenciação simbólica
  
  - Às vezes, você pode calcular a derivada em caneta e papel e potencialmente simplificar a expressão.
  - Com efeito, aplicações repetidas da regra da cadeia, regra do produto etc.
  - Às vezes, embora nem sempre, seja a opção mais precisa e rápida, se houver simplificações algébricas.
  - Às vezes, a integração simbólica no computador é uma boa solução, se o pacote puder lidar com suas funções. Fazer álgebra manualmente é tedioso e propenso a erros, e às vezes é inestimável.
    
- Diferenças finitas:  
  
  - Avalie a função pelo menos $ N $ para obter o gradiente – Jacobianos são ainda piores.  
  - $ \Delta $ Grande é numericamente estável, mas impreciso, $ \Delta $ muito pequeno, é numericamente instavel, porém mais preciso.
  - Evite se puder e use pacotes (por exemplo, [DiffEqDiffTools.jl](https://github.com/JuliaDiffEq/DiffEqDiffTools.jl) ) para obter uma boa escolha do $ \Delta $  
  - Se uma função é $ R^N \to R $ para um grande $ N $, isso exige avaliações da função $ O(N) $.


$$
\partial_{x_i}f(x_1,\ldots x_N) \approx \frac{f(x_1,\ldots x_i + \Delta,\ldots x_N) - f(x_1,\ldots x_i,\ldots x_N)}{\Delta}
$$


- Diferenciação automática. 
  
  - O mesmo que diferenciação analítica/simbólica, mas onde a **regra da cadeia** é calculada **numericamente** em vez de simbolicamente.
  - Assim como com derivadas analíticas, é possível estabelecer regras para derivadas de funções individuais (por exemplo, $ d \left (sin(x) \right) $ a $ cos (x) dx $) para derivadas intrínsecas.
  
A DA possui duas abordagens básicas, que são variações na ordem de avaliação da regra da cadeia: modo reverso e encaminhamento (embora o modo misto seja possível).

1. Se uma função é $ R^N \to R $, **modo reverso** DA pode encontrar o gradiente em $ O(1) $ sweep (onde uma "varredura" é a função $ O(1) $ avaliações).
1. Se uma função é $ R \to R ^ N $, o **modo de avanço** DA pode encontrar o jacobiano em $ O(1) $ sweeps.

Vamos explorar dois tipos de diferenciação automática em Julia (e discutir alguns pacotes que os implementam). Para ambos, lembre-se da [regra da cadeia](https://en.wikipedia.org/wiki/Chain_rule):

$$
\frac{dy}{dx} = \frac{dy}{dw} \cdot \frac{dw}{dx}
$$

O modo de avanço inicia o cálculo da esquerda com $ \frac{dy}{dw} $ primeiro, que calcula o produto com $ \frac {dw} {dx} $. Por outro lado, o modo reverso inicia no lado direito com $ \frac{dw}{dx} $ e trabalha para trás.

Tomemos um exemplo de uma função com operações fundamentais e derivadas analíticas conhecidas:

$$
f(x_1, x_2) = x_1 x_2 + \sin(x_1)
$$

E reescreva isso como uma função que contém uma sequência de operações simples e temporárias.

Iremos explorar os pacotes Diferenciação Automática (*DA*) em Julia ao invés dos alternativos.

In [3]:
function f(x_1, x_2)
    w_1 = x_1
    w_2 = x_2
    w_3 = w_1 * w_2
    w_4 = sin(w_1)
    w_5 = w_3 + w_4
    return w_5
end

f (generic function with 1 method)

Aqui podemos identificar todas as funções subjacentes (`*, sin, +`) e ver se cada uma possui uma
derivada intrínseca. Embora isso seja óbvio, com Julia poderíamos criar todo tipo de regras de diferenciação para arbitrariamente
combinações e composições complicadas de operações intrínsecas. De fato, há ainda [um pacote](https://github.com/JuliaDiff/ChainRules.jl) por registrar mais.

### Diferenciação automática no modo de avanço

Na *DA* de modo avançado, você primeiro corrige a variável na qual está interessado (chamado de "propagação") e depois avalia a regra da cadeia na ordem da esquerda para a direita.

Por exemplo, com o exemplo de $ f(x_1, f_2) $ acima, se quisermos calcular a derivada em relação a $ x_1 $,
podemos propagar a configuração de acordo.  $ \frac{\partial w_1}{\partial x_1} = 1 $ desde que estamos usando a derivada, enquanto $ \frac{\partial w_1}{\partial x_1} = 0 $.

Depois disso, refaça todos os cálculos da derivada em paralelo com a própria função.
$$
\begin{array}{l|l}
f(x_1, x_2) &
\frac{\partial f(x_1,x_2)}{\partial x_1}
\\
\hline
w_1 = x_1 &
\frac{\partial  w_1}{\partial  x_1} = 1 \text{ (seed)}\\
w_2 = x_2 &
\frac{\partial   w_2}{\partial  x_1} = 0 \text{ (seed)}
\\
w_3 = w_1 \cdot w_2 &
\frac{\partial  w_3}{\partial x_1} = w_2 \cdot \frac{\partial w_1}{\partial x_1} + w_1 \cdot \frac{\partial   w_2}{\partial  x_1}
\\
w_4 = \sin w_1 &
\frac{\partial   w_4}{\partial x_1} = \cos w_1 \cdot \frac{\partial  w_1}{\partial x_1}
\\
w_5 = w_3 + w_4 &
\frac{\partial  w_5}{\partial x_1} = \frac{\partial  w_3}{\partial x_1} + \frac{\partial  w_4}{\partial x_1}
\end{array}
$$

Como essas duas podem ser feitas ao mesmo tempo, dizemos que há "uma passagem" necessária para esse cálculo.

Generalizando um pouco, se a função tivesse valor vetorial, esse passe único obteria toda a linha do jacobiano nesse passe único. Portanto, para uma função $ R^N \to R^M $, são necessários passes de $ N $ para obter um jacobiano denso usando a DA de modo avançado.

Como você pode implementar a DA de modo avançado? É bastante fácil, com uma linguagem de programação genérica, dar um exemplo simples (enquanto o "diabo" está nos detalhes de
uma implementação de alto desempenho).

### Modo de avanço com números duplos

Uma maneira para implementar isso (usado no DA de modo avançado) é usar [números duplos](https://en.wikipedia.org/wiki/Dual_number).

Em vez de trabalhar com apenas um número real, por exemplo. $ x $, aumentaremos cada um com um infinitesimal $ \epsilon $ e usamos $ x + \epsilon $.

A partir do teorema de Taylor:

$$
f(x + \epsilon) = f(x) + f'(x)\epsilon + O(\epsilon^2)
$$

onde vamos definir o infinitesimal tal que $ \epsilon^2 = 0 $.

Com essa definição, podemos escrever uma regra geral para diferenciação de $ g (x, y) $ como a regra de cadeia para a derivada total:

$$
g(x + \epsilon, y + \epsilon) = g(x, y) + (\partial_x g(x,y) + \partial_y g(x,y))\epsilon
$$

Porém, observe que, se acompanharmos a constante na frente dos termos $ \epsilon $ (por exemplo, um $ x' $ e $ y' $).

$$
g(x + x'\epsilon, y + y'\epsilon) = g(x, y) + (\partial_x g(x,y)x' + \partial_y g(x,y)y')\epsilon
$$

Isso é simplesmente a regra da cadeia.  Um pouco mais de exemplos:

$$
\begin{aligned}
        (x + x'\epsilon) + (y + y'\epsilon) &= (x + y) + (x' + y')\epsilon\\
(x + x'\epsilon)\times(y + y'\epsilon) &= (xy) + (x'y + y'x)\epsilon\\
\exp(x + x'\epsilon) &= \exp(x) + (x'\exp(x))\epsilon\\
        \end{aligned}
$$

Usando a programação genérica em Julia, é fácil definir um novo tipo de número duplo que pode encapsular o par $ (x, x') $ e fornecer definições para todas as operações básicas. Cada definição possui a regra da cadeia incorporada.

Com essa abordagem, o processo “semente” é simples, a criação do $ \epsilon $ para a variável subjacente.

Portanto, se tivermos a função $ f(x_1, x_2) $ e desejarmos encontrar a derivada $ \partial_{x_1} f(3.8, 6.9) $ então os semearíamos com os números duplos $ x_1 \to (3.8, 1) $ e $ x_2 \to (6.9, 0) $.

Se você seguir todas as mesmas operações escalares acima com um número duplo inicial, ele calculará o valor da função e a derivada em uma única “varredura” e sem modificar nenhum código (genérico).

### ForwardDiff.jl

Os números duplos estão no coração de um dos pacotes da DA que já vimos.

In [4]:
using ForwardDiff
h(x) = sin(x[1]) + x[1] * x[2] + sinh(x[1] * x[2]) # multivariada.
x = [1.4 2.2]
@show ForwardDiff.gradient(h,x) # use DA, seeds de x

#Ou, pode usar funções complicadas de muitas variáveis
f(x) = sum(sin, x) + prod(tan, x) * sum(sqrt, x)
g = (x) -> ForwardDiff.gradient(f, x); # g() é agora o gradiente
g(rand(5)) # gradiente em um ponto aleatório
# ForwardDiff.hessian(f,x') # ou o hessiano

ForwardDiff.gradient(h, x) = [26.354764961030977 16.663053156992284]


5-element Array{Float64,1}:
 1.818314452615056 
 2.212251895373916 
 2.461284516606687 
 1.9514591707639093
 1.8393629512203116

Podemos até diferenciar automaticamente funções complicadas com iterações incorporadas.

In [5]:
function squareroot(x) #fingindo que não sabemos sqrt()
    z = copy(x)  #Ponto inicial do método de Newton
    while abs(z*z - x) > 1e-13
        z = z - (z*z-x)/(2z)
    end
    return z
end
squareroot(2.0)

1.4142135623730951

In [6]:
using ForwardDiff
dsqrt(x) = ForwardDiff.derivative(squareroot, x)
dsqrt(2.0)

0.35355339059327373

### Zygote.jl

Diferentemente da diferenciação automática no modo avançado, o modo reverso é muito difícil de implementar com eficiência, e há muitas variações na melhor abordagem.

Muitos pacotes de modo reverso estão conectados a pacotes de aprendizado de máquina, pois os gradientes eficientes das funções de perda de $ R^N \to R $ são necessários para os algoritmos de otimização de descida de gradiente usados no aprendizado de máquina.

Um pacote recente é [Zygote.jl](https://github.com/FluxML/Zygote.jl), que é usado na estrutura Flux.jl.

In [7]:
using Zygote

h(x, y) = 3x^2 + 2x + 1 + y*x - y
gradient(h, 3.0, 5.0)

(25.0, 2.0)

Aqui vemos que o Zygote tem uma função de gradiente como interface, que retorna uma tupla.

Você pode criar isso como um operador se quiser.,

In [8]:
D(f) = x-> gradient(f, x)[1]  # Retorna a primera na tupla

D_sin = D(sin)
D_sin(4.0)

-0.6536436208636119

Para funções de uma variável (Julia), podemos encontrá-lo simplesmente usando o `'` após o nome de uma função.

In [9]:
using Statistics
p(x) = mean(abs, x)
p'([1.0, 3.0, -2.0])

3-element Array{Float64,1}:
  0.3333333333333333
  0.3333333333333333
 -0.3333333333333333

Ou, usando a função iterativa complicada que definimos para o quadrado.

In [10]:
squareroot'(2.0)

0.3535533905932737

O Zygote suporta combinações de vetores e escalares como parâmetros de função.

In [11]:
h(x,n) = (sum(x.^n))^(1/n)
gradient(h, [1.0, 4.0, 6.0], 2.0)

([0.13736056394868904, 0.5494422557947561, 0.8241633836921343], -1.2725553130925444)

Os gradientes podem ter dimensões muito altas. Por exemplo, para resolver um problema simples de otimização não linear
com 1 milhão de dimensões, resolvidas em alguns segundos.

In [12]:
using Optim, LinearAlgebra
N = 1000000
y = rand(N)
λ = 0.01
obj(x) = sum((x .- y).^2) + λ*norm(x)

x_iv = rand(N)
function g!(G, x)
    G .=  obj'(x)
end

results = optimize(obj, g!, x_iv, LBFGS()) # ou ConjugateGradient()
println("minimum = $(results.minimum) with in "*
"$(results.iterations) iterations")

minimum = 5.773501129493461 with in 2 iterations


Cuidado: embora o Zygote seja a implementação mais interessante da DA de modo reverso em Julia, ele possui muitas arestas.

- Se você escreve uma função, pega seu gradiente e modifica a função, é necessário chamar `Zygote.refresh ()` ou o gradiente ficará fora de sincronia. Isso pode não se aplicar ao Julia 1.3 ou superior.
- Ele não fornece recursos para obter jacobianos, portanto, você deve solicitar cada linha do jacobiano separadamente. Dito isto, você
  provavelmente deseja usar o `ForwardDiff.jl` para jacobianos se a dimensão da saída for semelhante à dimensão da entrada.
- Você não pode, no release atual, usar funções de mutação (por exemplo, modificar um valor em uma matriz, etc.), Embora esse recurso esteja em andamento.
- A compilação pode ser muito lenta para funções complicadas.  

## Otimização

Há um grande número de pacotes destinados a serem usadospara otimização na Julia.

Parte do motivo da diversidade de opções é que Julia possibilita a implementação eficiente de um grande número de variações nas rotinas de otimização.

A outra razão é que tipos diferentes de problemas de otimização requerem algoritmos diferentes.

### Optim.jl

Uma boa solução de Julia pura para a otimização (sem restrições ou limitada) da função univariada e multivariada é o pacote [Optim.jl](https://github.com/JuliaNLSolvers/Optim.jl).

Por padrão, os algoritmos no `Optim.jl` levam a minimização do destino em vez de maximização, portanto, se uma função é chamada, `optimize` isso significa minimização.

#### Funções Univariadas em Intervalos limitados

O padrão de [Otimização Univariada](http://julianlsolvers.github.io/Optim.jl/stable/#user/minimization/#minimizing-a-univariate-function-on-a-bounded-interval)
é uma rotina robusta de otimização híbrida chamada [método de Brent.](https://en.wikipedia.org/wiki/Brent%27s_method).

In [13]:
using Optim
using Optim: converged, maximum, maximizer, minimizer, iterations #algumas funções extras

result = optimize(x-> x^2, -2.0, 1.0)

Results of Optimization Algorithm
 * Algorithm: Brent's Method
 * Search Interval: [-2.000000, 1.000000]
 * Minimizer: 0.000000e+00
 * Minimum: 0.000000e+00
 * Iterations: 5
 * Convergence: max(|x - x_upper|, |x - x_lower|) <= 2*(1.5e-08*|x|+2.2e-16): true
 * Objective Function Calls: 6

Sempre verifique se os resultados convergiram, e gere os erros caso contrário.

In [14]:
converged(result) || error("Failed to converge in $(iterations(result)) iterations")
xmin = result.minimizer
result.minimum

0.0

A primeira linha é um OR lógico entre `converged(result)` e `error("...")`.

Se a verificação de convergência for aprovada, a sentença lógica será verdadeira e prosseguirá para a próxima linha; caso contrário, lançará o erro.

Ou para maximizar.

In [15]:
f(x) = -x^2
result = maximize(f, -2.0, 1.0)
converged(result) || error("Failed to converge in $(iterations(result)) iterations")
xmin = maximizer(result)
fmax = maximum(result)

-0.0

**Nota:** Perceba que chamamos resultados `optimize` usando `result.minimizer`, e resultados para `maximize` usando `maximizer(result)`.

#### Otimização Multivariada Irrestrita

Há uma variedade de [algorítimos e opções](http://julianlsolvers.github.io/Optim.jl/stable/#user/minimization/#_top) para otimização multivaridada.

A partir da documentação, a versão mais simples é:

In [16]:
f(x) = (1.0 - x[1])^2 + 100.0 * (x[2] - x[1]^2)^2
x_iv = [0.0, 0.0]
results = optimize(f, x_iv) # por exemplo. optimize(f, x_iv, NelderMead())

 * Status: success

 * Candidate solution
    Minimizer: [1.00e+00, 1.00e+00]
    Minimum:   3.525527e-09

 * Found with
    Algorithm:     Nelder-Mead
    Initial Point: [0.00e+00, 0.00e+00]

 * Convergence measures
    √(Σ(yᵢ-ȳ)²)/n ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    60
    f(x) calls:    118


O algorítimo padrão em `NelderMead`, que é livre de derivadas e, portanto, requer muitas avaliações de função.

Para mudar o tipo do algorítimo para [L-BFGS](http://julianlsolvers.github.io/Optim.jl/stable/#algo/lbfgs/).

In [17]:
results = optimize(f, x_iv, LBFGS())
println("minimum = $(results.minimum) with argmin = $(results.minimizer) in "*
"$(results.iterations) iterations")

minimum = 5.3784046148998115e-17 with argmin = [0.9999999926662393, 0.9999999853324786] in 24 iterations


Observe que isso possui menos iterações.

Como nenhuma derivada foi fornecida, utilizou [diferenças finitas](https://en.wikipedia.org/wiki/Finite_difference) para aproximar o gradiente de `f(x)`.

No entanto,  como a maioria dos algoritmos exige derivadas, você frequentemente desejará usar a diferenciação automática ou passar gradientes analíticos, se possível.

In [18]:
f(x) = (1.0 - x[1])^2 + 100.0 * (x[2] - x[1]^2)^2
x_iv = [0.0, 0.0]
results = optimize(f, x_iv, LBFGS(), autodiff=:forward) # por exemplo use ForwardDiff.jl
println("minimum = $(results.minimum) with argmin = $(results.minimizer) in "*
"$(results.iterations) iterations")

minimum = 5.191703158437428e-27 with argmin = [0.999999999999928, 0.9999999999998559] in 24 iterations


Note que não precisamos usar diretamente o `ForwardDiff.jl`, desde que nossa função `f(x)` foi escrita para ser genérica (veja a [aula de programação genérica](https://lectures.quantecon.org/generic_programming.html)).

Alternativamente, com um gradiente analítico.

In [19]:
f(x) = (1.0 - x[1])^2 + 100.0 * (x[2] - x[1]^2)^2
x_iv = [0.0, 0.0]
function g!(G, x)
    G[1] = -2.0 * (1.0 - x[1]) - 400.0 * (x[2] - x[1]^2) * x[1]
    G[2] = 200.0 * (x[2] - x[1]^2)
end

results = optimize(f, g!, x_iv, LBFGS()) # ou ConjugateGradient()
println("minimum = $(results.minimum) with argmin = $(results.minimizer) in "*
"$(results.iterations) iterations")

minimum = 5.191703158437428e-27 with argmin = [0.999999999999928, 0.9999999999998559] in 24 iterations


Para métodos sem derivadas, você pode alterar o algoritmo - e não precisa fornecer um gradiente.

In [20]:
f(x) = (1.0 - x[1])^2 + 100.0 * (x[2] - x[1]^2)^2
x_iv = [0.0, 0.0]
results = optimize(f, x_iv, SimulatedAnnealing()) # ou ParticleSwarm() ou NelderMead()

 * Status: failure (reached maximum number of iterations) (line search failed)

 * Candidate solution
    Minimizer: [7.79e-01, 6.00e-01]
    Minimum:   5.355153e-02

 * Found with
    Algorithm:     Simulated Annealing
    Initial Point: [0.00e+00, 0.00e+00]

 * Convergence measures
    |x - x'|               = NaN ≰ 0.0e+00
    |x - x'|/|x'|          = NaN ≰ 0.0e+00
    |f(x) - f(x')|         = NaN ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = NaN ≰ 0.0e+00
    |g(x)|                 = NaN ≰ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    1000
    f(x) calls:    1001


No entanto, você observará que isso não convergiu, pois os métodos estocásticos geralmente exigem muito mais iterações como uma troca por suas propriedades de convergência global.

Veja o exemplo de [probabilidade máxima](http://julianlsolvers.github.io/Optim.jl/stable/#examples/generated/maxlikenlm/)
e [notebook Jupyter](https://nbviewer.jupyter.org/github/JuliaNLSolvers/Optim.jl/blob/gh-pages/v0.15.3/examples/generated/maxlikenlm.ipynb) que o acompanha.

### JuMP.jl

O pacote  [JuMP.jl](https://github.com/JuliaOpt/JuMP.jl) é uma ambiciosa implementação de uma linguagem de modelagem para problemas de otimização em Julia.

Nesse sentido, é mais parecido com um AMPL (ou Pyomo) construído sobre a linguagem Julia com macros e capaz de usar uma variedade de diferentes solucionadores comerciais e de código aberto.

Se você tiver um problema linear, quadrático, cônico, linear inteiro misto, etc. esse provavelmente será o “mega-pacote” ideal para chamar vários solucionadores.

Para problemas não-lineares, a linguagem de modelagem pode dificultar as funções complicadas (pois não foi projetada para ser usada como um otimizador não-linear de uso geral).

Veja o [guia de inicio rápido](http://www.juliaopt.org/JuMP.jl/0.18/quickstart.html) para mais detalhes em todas as opções.

A seguir, é apresentado um exemplo de como chamar um objetivo linear com uma restrição não linear (fornecida por uma função externa).

Aqui `Ipopt` apoia `Interior Point OPTimizer`, [solucinador](https://github.com/JuliaOpt/Ipopt.jl) em Julia.

In [21]:
using JuMP, Ipopt
# Resolva
# max( x[1] + x[2] )
# st sqrt(x[1]^2 + x[2]^2) <= 1

function squareroot(x) #fingindo que não sabemos sqrt()
    z = x #  Ponto de início inicial do método de Newton
    while abs(z*z - x) > 1e-13
        z = z - (z*z-x)/(2z)
    end
    return z
end
m = Model(with_optimizer(Ipopt.Optimizer))
# precisa registrar funções definidas pelo usuário para o DA
JuMP.register(m,:squareroot, 1, squareroot, autodiff=true)

@variable(m, x[1:2], start=0.5) # start é a condição inicial 
@objective(m, Max, sum(x))
@NLconstraint(m, squareroot(x[1]^2+x[2]^2) <= 1)
@show JuMP.optimize!(m)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        2
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equ

E este é um exemplo de objetivo quadrático.

In [22]:
# resolva
# min (1-x)^2 + 100(y-x^2)^2)
# st x + y >= 10

using JuMP,Ipopt
m = Model(with_optimizer(Ipopt.Optimizer)) # configurações para a solução
@variable(m, x, start = 0.0)
@variable(m, y, start = 0.0)

@NLobjective(m, Min, (1-x)^2 + 100(y-x^2)^2)

JuMP.optimize!(m)
println("x = ", value(x), " y = ", value(y))

# adicionando uma restrição (linear)
@constraint(m, x + y == 10)
JuMP.optimize!(m)
println("x = ", value(x), " y = ", value(y))

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

### BlackBoxOptim.jl

Outro pacote para a otimização global sem derivadas é o [BlackBoxOptim.jl](https://github.com/robertfeldt/BlackBoxOptim.jl).

Para ver um exemplo da documentação:

In [23]:
using BlackBoxOptim

function rosenbrock2d(x)
return (1.0 - x[1])^2 + 100.0 * (x[2] - x[1]^2)^2
end

results = bboptimize(rosenbrock2d; SearchRange = (-5.0, 5.0), NumDimensions = 2);

Starting optimization with optimizer DiffEvoOpt{FitPopulation{Float64},RadiusLimitedSelector,BlackBoxOptim.AdaptiveDiffEvoRandBin{3},RandomBound{ContinuousRectSearchSpace}}
0.00 secs, 0 evals, 0 steps

Optimization stopped after 10001 steps and 0.09 seconds
Termination reason: Max number of steps (10000) reached
Steps per second = 112462.99
Function evals per second = 114228.49
Improvements/step = 0.21650
Total function evaluations = 10158


Best candidate found: [1.0, 1.0]

Fitness: 0.000000000



Um exemplo para  [execução paralela](https://github.com/robertfeldt/BlackBoxOptim.jl/blob/master/examples/rosenbrock_parallel.jl) do objetivo é fornecido.

## Sistema de Equações e Mínimos Quadrados

### Roots.jl

A raiz de uma função real $ f $ em $ [a,b] $ é um $ x \in [a, b] $ tal que $ f(x)=0 $

Por exemplo, se plotarmos a função:


<a id='equation-root-f'></a>
$$
f(x) = \sin(4 (x - 1/4)) + x + x^{20} - 1 \tag{1}
$$

com $ x \in [0,1] $ nós obtemos:

<img src="https://julia.quantecon.org/more_julia/_static/figures/sine-screenshot-2.png" style="width:50%;">

  
A única raiz aproximada é 0.408.

O pacote [Roots.jl](https://github.com/JuliaLang/Roots.jl) oferece `fzero()` para encontrar as raizes.

In [24]:
using Roots
f(x) = sin(4 * (x - 1/4)) + x + x^20 - 1
fzero(f, 0, 1)

0.40829350427936706

### NLsolve.jl

O pacote [NLsolve.jl](https://github.com/JuliaNLSolvers/NLsolve.jl/) fornece funções para resolver sistemas multivariados e equações e pontos fixos.

A partir da documentação, resolver um sistema de equações sem fornecer um método jacobiano.

In [25]:
using NLsolve

f(x) = [(x[1]+3)*(x[2]^3-7)+18
        sin(x[2]*exp(x[1])-1)] # retorna um array

results = nlsolve(f, [ 0.1; 1.2])

Results of Nonlinear Solver Algorithm
 * Algorithm: Trust-region with dogleg and autoscaling
 * Starting Point: [0.1, 1.2]
 * Zero: [-7.775508345910301e-17, 0.9999999999999999]
 * Inf-norm of residuals: 0.000000
 * Iterations: 4
 * Convergence: true
   * |x - x'| < 0.0e+00: false
   * |f(x)| < 1.0e-08: true
 * Function Calls (f): 5
 * Jacobian Calls (df/dx): 5

No caso acima, o algoritmo usou diferenças finitas para calcular o jacobiano.

Como alternativa, se `f(x)` for escrito genericamente, você poderá usar a diferenciação automática com uma única configuração.

In [26]:
results = nlsolve(f, [ 0.1; 1.2], autodiff=:forward)

println("converged=$(NLsolve.converged(results)) at root=$(results.zero) in "*
"$(results.iterations) iterations and $(results.f_calls) function calls")

converged=true at root=[-3.487552479724522e-16, 1.0000000000000002] in 4 iterations and 5 function calls


Fornecer uma função que opera no local (isto é, modifica um argumento) pode ajudar o desempenho de grandes sistemas de equações (e prejudicá-lo para os pequenos).

In [27]:
function f!(F, x) # modifica o primeiro argumento
    F[1] = (x[1]+3)*(x[2]^3-7)+18
    F[2] = sin(x[2]*exp(x[1])-1)
end

results = nlsolve(f!, [ 0.1; 1.2], autodiff=:forward)

println("converged=$(NLsolve.converged(results)) at root=$(results.zero) in "*
"$(results.iterations) iterations and $(results.f_calls) function calls")

converged=true at root=[-3.487552479724522e-16, 1.0000000000000002] in 4 iterations and 5 function calls


## LeastSquaresOptim.jl

Muitos problemas de otimização podem ser resolvidos usando mínimos quadrados lineares ou não lineares.

Seja $ x \in R^N $ e $ F(x) : R^N \to R^M $ com $ M \geq N $, então o problema de mínimos quadrados não lineares é:

$$
\min_x F(x)^T F(x)
$$

Enquanto $ F(x)^T F(x) \to R $, e portanto esse problema pode tecnicamente usar qualquer otimizador não linear, é usual explorar a estrutura do problema.

Em particular, o Jacobiano de $ F(x) $, pode ser usado para aproximar o Hessiano do objetivo.

Como na maioria dos problemas de otimização não linear, os benefícios geralmente se tornam evidentes somente quando a diferenciação analítica ou automática é possível.

Se $ M = N $ e conhecemos uma raiz $ F(x^*) = 0 $ o sistema de equações existe, então NLS (non linear least squares) 
é o método de fato para resolver grandes **sistemas de equações**.

Uma implementação do NLS é fornecida em [LeastSquaresOptim.jl](https://github.com/matthieugomez/LeastSquaresOptim.jl).

A partir da documentação.

In [28]:
using LeastSquaresOptim
function rosenbrock(x)
    [1 - x[1], 100 * (x[2]-x[1]^2)]
end
LeastSquaresOptim.optimize(rosenbrock, zeros(2), Dogleg())

Results of Optimization Algorithm
 * Algorithm: Dogleg
 * Minimizer: [1.0,1.0]
 * Sum of squares at Minimum: 0.000000
 * Iterations: 51
 * Convergence: true
 * |x - x'| < 1.0e-08: false
 * |f(x) - f(x')| / |f(x)| < 1.0e-08: true
 * |g(x)| < 1.0e-08: false
 * Function Calls: 52
 * Gradient Calls: 36
 * Multiplication Calls: 159


**Nota:** Como existe um conflito de nomes entre `Optim.jl` e esse pacote, para ambos precisamos qualificar o uso da função `optimize` (por exemplo, `LeastSquaresOptim.optimize`).

Aqui, por padrão, ele usará a *DA* com `ForwardDiff.jl` para calcular o Jacobiano,
mas você também poderá fornecer seu próprio cálculo do jacobiano (analítico ou usando diferenças finitas) e/ou calcular a função no local.

In [29]:
function rosenbrock_f!(out, x)
    out[1] = 1 - x[1]
    out[2] = 100 * (x[2]-x[1]^2)
end
LeastSquaresOptim.optimize!(LeastSquaresProblem(x = zeros(2),
                                f! = rosenbrock_f!, output_length = 2))

# Se quiser use o gradiente
function rosenbrock_g!(J, x)
    J[1, 1] = -1
    J[1, 2] = 0
    J[2, 1] = -200 * x[1]
    J[2, 2] = 100
end
LeastSquaresOptim.optimize!(LeastSquaresProblem(x = zeros(2),
                                f! = rosenbrock_f!, g! = rosenbrock_g!, output_length = 2))

Results of Optimization Algorithm
 * Algorithm: Dogleg
 * Minimizer: [1.0,1.0]
 * Sum of squares at Minimum: 0.000000
 * Iterations: 51
 * Convergence: true
 * |x - x'| < 1.0e-08: false
 * |f(x) - f(x')| / |f(x)| < 1.0e-08: true
 * |g(x)| < 1.0e-08: false
 * Function Calls: 52
 * Gradient Calls: 36
 * Multiplication Calls: 159


## Notas Adicionais

Assista a [esse vídeo](https://www.youtube.com/watch?v=vAp6nUMrKYg&feature=youtu.be) de um dos criadores de Julia sobre diferenciação automática.

## Exercícios

### Exercício 1

A execução simples de diferenciação automática de modo avançado é muito fácil em Julia, pois é genérica. Neste exercício, você
preencherá algumas das operações necessárias para uma implementação simples da DA.

Primeiro, precisamos fornecer um tipo para armazenar o dual.

In [30]:
struct DualNumber{T} <: Real
    val::T
    ϵ::T
end

Aqui nós o tornamos um subtipo de `Real` para que ele possa passar por funções que esperam reais.

Podemos adicionar uma variedade de definições de regra de cadeia importando nas funções apropriadas e adicionando versões DualNumber. Por exemplo:

In [31]:
import Base: +, *, -, ^, exp
+(x::DualNumber, y::DualNumber) = DualNumber(x.val + y.val, x.ϵ + y.ϵ)  #  adição dual
+(x::DualNumber, a::Number) = DualNumber(x.val + a, x.ϵ)  # ou seja, adição escalar, não dual
+(a::Number, x::DualNumber) = DualNumber(x.val + a, x.ϵ)  # ou seja, adição escalar, não dual

+ (generic function with 265 methods)

Com isso, podemos semear um número duplo e encontrar derivadas simples,

In [32]:
f(x, y) = 3.0 + x + y

x = DualNumber(2.0, 1.0)  # x -> 2.0 + 1.0 \epsilon
y = DualNumber(3.0, 0.0)  # ou seja. y = 3.0, sem derivada


# seeded calcula a função e o gradiente d / dx!
f(x,y)

DualNumber{Float64}(8.0, 1.0)

Para essa tarefa:

1. Adicione regras da DA para as outras operações: `*, -, ^, exp`.
1. Crie alguns exemplos de funções univariadas e multivariadas que combinam essas operações e use sua implementação da DA para encontrar as derivadas.